<a href="https://colab.research.google.com/github/basselkassem/predict_income_potential/blob/master/etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Transform Load (ETL) [coursera]

ETL is one of the first things which needs to be done in a data science project. 

The nature of this task highly depends on the type of data source. Whether it is relational or unstructured, enterprise data or internet data, persistent data or streaming data. This heavily influences the choice of architecture. Therefore, you must document your choice and thinking process in the Architectural Decision Document (ADD).

This task involves – as the name implies – accessing the data source, transforming it in a way it can be easily worked with and finally make it available to downstream analytics processes – either real-time streaming or batch ones.

- In case of operational relational data, de-normalization usually needs to take place
- for unstructured data, some feature extraction might already be appropriate
- for real-time data, windows are usually created.

# Downloading data

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data

--2019-10-26 21:49:33--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3974305 (3.8M) [application/x-httpd-php]
Saving to: ‘adult.data’

adult.data          100%[===================>]   3.79M  7.34MB/s    in 0.5s    

2019-10-26 21:49:34 (7.34 MB/s) - ‘adult.data’ saved [3974305/3974305]



In [3]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test

--2019-10-26 21:49:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2003153 (1.9M) [application/x-httpd-php]
Saving to: ‘adult.test’

adult.test          100%[===================>]   1.91M  4.40MB/s    in 0.4s    

2019-10-26 21:49:35 (4.40 MB/s) - ‘adult.test’ saved [2003153/2003153]



In [4]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names

--2019-10-26 21:49:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5229 (5.1K) [application/x-httpd-php]
Saving to: ‘adult.names’

adult.names         100%[===================>]   5.11K  --.-KB/s    in 0s      

2019-10-26 21:49:37 (152 MB/s) - ‘adult.names’ saved [5229/5229]



In [5]:
!ls

adult.data  adult.names  adult.test  sample_data


In [6]:
!cat adult.names

| This data was extracted from the census bureau database found at
| http://www.census.gov/ftp/pub/DES/www/welcome.html
| Donor: Ronny Kohavi and Barry Becker,
|        Data Mining and Visualization
|        Silicon Graphics.
|        e-mail: ronnyk@sgi.com for questions.
| Split into train-test using MLC++ GenCVFiles (2/3, 1/3 random).
| 48842 instances, mix of continuous and discrete    (train=32561, test=16281)
| 45222 if instances with unknown values are removed (train=30162, test=15060)
| Duplicate or conflicting instances : 6
| Class probabilities for adult.all file
| Probability for the label '>50K'  : 23.93% / 24.78% (without unknowns)
| Probability for the label '<=50K' : 76.07% / 75.22% (without unknowns)
|
| Extraction was done by Barry Becker from the 1994 Census database.  A set of
|   reasonably clean records was extracted using the following conditions:
|   ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
|
| Prediction task is to determine whether a person makes over

# Importing libs

In [0]:
import pandas as pd

# Reading Data


The downloaded data is a tabular data that exists in tow files with comma separated format. We are going to:
- Assign names to the columns
- Read data form the 2 files and merge them
- Save the merged data to IBM object storage



In [8]:
data_headers = ['age', 'workclass', 'final_weight', 'education', 'education_duration', 
                'marital_status', 'occupation', 'relationship', 'race', 'sex', 
                'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'target']

print('Data attributes: ', len(data_headers))

Data attributes:  15


In [0]:
part1 = pd.read_csv('adult.data', header = None, names = data_headers)
part2 = pd.read_csv('adult.test', header = None, skiprows = 1, names = data_headers)

In [10]:
print('First part of the data: ', part1.shape)
print('Second part of the data: ', part2.shape)
df = pd.concat([part1, part2], ignore_index = True, axis = 0)
print('total size of the data:', df.shape)

First part of the data:  (32561, 15)
Second part of the data:  (16281, 15)
total size of the data: (48842, 15)


In [11]:
df.head()

,age,workclass,final_weight,education,education_duration,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,target
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data_path = "/content/gdrive/My Drive/Colab Notebooks/predict_income_potential/data/"
df.to_csv(data_path + "init_data.csv", index = 0)